# **Application à la génétique**

Dans les questions suivantes, nous étudierons un exemple d'application de réseaux convolutionels à la biologie (pas de panique, nul besoin de prérequis en biologie).
L'acide désoxyribonucléique (ADN), présent dans toutes les cellules de votre corps, est essentiel à la conception des protéines qui lui sont utiles. L'ADN peut être représenté par une longue chaîne de caractères, formée par l'alphabet \{A,C,T,G\}.
La seule propriété de l'ADN qui nous intéresse ici est que toutes les régions ne sont pas accessibles aux molécules extérieures (~98% de l'ADN n'est pas accessible). 
Comprendre quelles parties de l'ADN sont accessibles et pourquoi est d'intérêt crucial pour les scientifiques. Cela motive l'apprentissage de modèles capable de classifier ces parties-là. 
Dans ce devoir, nous reproduirons un réseau profond appelé [Basset](https://pubmed.ncbi.nlm.nih.gov/27197224/). Il s'agit d'une méthode de classification qui apprend quelles régions d'ADN sont accessibles.

In [ ]:
#@title Connecter votre Google Drive
%matplotlib inline
%load_ext autoreload
%autoreload 2

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#@title Lier le dossier du devoir & les installations requises
#@markdown Entrer le chemin vers le dossier du devoir dans votre Google Drive
import sys
import os
import shutil
import warnings

folder = "/content/gdrive/MyDrive/A1" #@param {type:"string"}
!ln -Ts $folder /content/A1 2> /dev/null

# Add the assignment folder to Python path
if '/content/A1' not in sys.path:
    sys.path.insert(0, '/content/A1')

# Install requirements
!pip install -qr /content/A1/requirements.txt

# Check if CUDA is available
import torch
if not torch.cuda.is_available():
    warnings.warn('CUDA is not available.')

In [ ]:
# unzip data
if not os.path.exists('/content/A1/er.h5'):
    os.system('unzip /content/A1/er.zip')
    os.system('mv /content/er.h5 /content/A1/er.h5')

## Utiliser un GPU
Dans Google Colab, vous pouver lancer votre code sur GPU. Cela est particulièrement important pour la partie CNN du devoir. Pour s'assurer que le notebook utilise un GPU, vous pouvez changer les paramètres du notebook qu'on pour retrouver ainsi
* (EN) `Edit > Notebook Settings`
* (FR) `Modifier > Paramètres du notebook`

Nul besoin d'utiliser de GPU si votre code n'en a pas besoin.

In [ ]:
import random

import numpy as np
import h5py
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

import solution

In [ ]:
# The hyperparameters we will use
batch_size = 64
learning_rate = 0.002

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# set RNG
seed = 42
torch.backends.cudnn.deterministic = True
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
if device.type=='cuda':
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

Avant d'entrainer le classifier, inspectons les données que nous utiliserons plus tard. Ceci est une bonne habitude à garder (surtout si les données sortent de l'ordinaire) !

Dans notre cas, les données d'entrées sont des séquences d'ADN qui viennent de la référence du génome humain HG19: il est considéré comme le prototype de l'ADN humain, mais il ne correspond à aucune personne en précis. 

Dans notre cas, les données génétiques sont encodées en *one-hot*, c'est à dire:
$$A = \begin{pmatrix} 1 \\ 0 \\ 0 \\ 0 \end{pmatrix}, C = \begin{pmatrix} 0 \\ 1 \\ 0 \\ 0 \end{pmatrix}, G = \begin{pmatrix} 0 \\ 0 \\ 1 \\ 0 \end{pmatrix}, T = \begin{pmatrix} 0 \\ 0 \\ 0 \\ 1 \end{pmatrix}$$

Chaque caractère $\{A,C,G,T\}$ est représenté par un vecteur, qu'on appellera paire de bases. Ces vecteurs sont concaténés le long de la séquence définie par la chaine de caractères de l'ADN. Ainsi chaque échantillon est équivalent à une image de taille (longueur de la séquence, 4) et une seule chaîne.

Les cibles de la prédiction sont binaires et indiquent si l'ADN est accessible ou pas pour un ensemble de 164 différentes expériences. Pour chaque échantillon, la cible est donc un vecteur de taille 164, avec 1 aux indices des expériences pour lesquelles l'ADN est accessible et 0 ailleurs.

# **Question 1** (Comprendre les données). 
(10 points) Vous aurez probablement not. que PyTorch est orienté objet -- tout est une sous-classe d'une classe Pytorch. Pour les *datasets*, on hérite de la classe `torch.utils.data.Dataset`.

1.   Définir les méthodes `__getitem__` and `__len__` pour `utils.BassetDataset` dans le fichier `solution.py`.
2.   Quelle est la longueur de chaque séquence présente dans ce dataset ? Ces séquences seront présentées comme entrées de notre modèle plus tard.
3.   Peut-on considérer que chaque échantillon peut aussi être une image de taille (1, longueur de la séquence) avec 4 chaînes?

N.B.: Ces données ont déjà été pré-processées. Nul besoin de les retoucher ou transformer à ce stade !

In [ ]:
# investigate your data
f = h5py.File('/content/A1/er.h5', 'r')
f.keys()
f.close()

In [ ]:
basset_dataset_train = solution.BassetDataset(path='/content/A1', f5name='er.h5', split='train')
basset_dataset_valid = solution.BassetDataset(path='/content/A1', f5name='er.h5', split='valid')
basset_dataset_test = solution.BassetDataset(path='/content/A1', f5name='er.h5', split='test')
basset_dataloader_train = DataLoader(basset_dataset_train,
                                     batch_size=batch_size,
                                     drop_last=True,
                                     shuffle=True,
                                     num_workers=1)
basset_dataloader_valid = DataLoader(basset_dataset_valid,
                                     batch_size=batch_size,
                                     drop_last=True,
                                     shuffle=False,
                                     num_workers=1)
basset_dataloader_test = DataLoader(basset_dataset_test,
                                    batch_size=batch_size,
                                    drop_last=True,
                                    shuffle=False,
                                    num_workers=1)

# **Question 2** (Construction du réseau)
(15 points) Comme pour l'objet du dataset, vous allez créer une sous-classe de la classe `nn.Module` qui définira Basset. La définition de cette sous-classe sera fait dans `solution.py`.

1. Remplir les valeurs manquantes (notées par `?` dans la définition de la classe `Basset` dans le ficher `utils.py`, en vous référant à la [figure 13](https://genome.cshlp.org/content/suppl/2016/06/10/gr.200535.115.DC1/Supplementary_Figures.pdf) et l'arithmétique des convolutions.
2. Écrire la fonction `forward`. Vous pouvez toujours suivre le modèle décrit par la [figure 13](https://genome.cshlp.org/content/suppl/2016/06/10/gr.200535.115.DC1/Supplementary_Figures.pdf).

N'incluez pas les sorties d'activation dans votre fonction `forward`. En pratique, il est mieux d'utiliser une fonction de coût ayant les sorties d'activation intégrées.

In [ ]:
model = solution.Basset().to(device)

# **Question 3 (Aire sous la courbe)** 
(25 points). 
Chaque séquence d'ADN est soit exposée (ce cas est qualifié de positif) ou pas (ce cas est qualifié de négatif). La sortie de notre modèle est utilisé pour produire une prédiction binaire concernant l'ADN d'entrée.

Pour un échantillon donnée, on dit que notre modèle a produit une prédiction positive quand la valeur de la sortie correspondante est au-dessus d'un certain seuil $k$. Dans le cas contraire, la prédiction est dite négative.

On définit comme *taux de vrais positifs* (TVP) le nombre de cas positifs correctement prédits divisé par le nombre de cas positifs. On définit comme *taux de faux positifs* (TFP) le nombre de cas incorrectement prédits comme positifs divisé par le nombre de cas négatifs.


1. Compléter la fonction `utils.compute_fpr_tpr` dans `solution.py`. Ce code calculera les taux TVP et TFP for étant donné les prédictions de votre modèle et les vraies cibles.

2. Les TFP et TVP changent en fonction de $k$. On peut visualiser cette dépendance en représentant le TFP en axe des x est le TVP en axe des y pour différentes valeurs de $k$; la courbe obtenue est appelée la courbe ROC. La forme de cette courbe nous informe sur la performance de nos classifieurs. Regardons à présent à quoi ressemble cette courbe quand notre modèle est encore aléatoire (pas encore entraîné). Dans `solution.py`, compléter la fonction `compute_fpr_tpr_dumb_model`. Cette fonction va générer 1000 valeurs aléatoires binaires qui seront utilisées comment nos cibles, et 1000 valeurs aléatoires uniformément entre 0 et 1 qui correspondront aux prédictions de notre modèle. La fonction calculera ensuite les TFP et TVP pour $k\in \{0, 0.05,..., 0.95\}$. Vous devriez générer la courbe ROC pour votre propre connaissance.

In [ ]:
solution.compute_fpr_tpr_dumb_model()

3. We will now simulate a better model. In `solution.py`, fill in the function `compute_fpr_tpr_smart_model`. This will simulate 1000 targets the same way as before. However, this will simulate model outputs as uniform random variables between 0.4 and 1 for the positive cases. For the negative cases, use uniform random variables between 0 and 0.6. Compute the tpr and fpr varying $k$ like before. You should also look at the ROC curve.

3. Nous allons simuler un modèle plus performant. In `solution.py`, compléter la fonction `compute_fpr_tpr_smart_model`. Cette fonction générera 1000 cibles comme précédemment. Cette fois, pour les cas positifs, elle simulera les sorties du modèles en générant uniformément des valeurs aléatoires entre 0.4 et 1. Pour les cas négatifs, les sorties du modèles seront simulées en générant uniformément des valeurs aléatoires 0 and 0.6. Calculer les TFP et TVP pour différents $k$ comme dans précédemment. Vous devriez générer la courbe ROC pour votre propre connaissance.

In [ ]:
solution.compute_fpr_tpr_smart_model()

5. L'aire sous la courbe (ASC) résume la courbe ROC en une seule valeur.
Celle-ci est calculée comme l'air sous la courbe ROC (vous pouvez utiliser l'approximation d'une surface sous une courbe par des rectangles). Compléter la fonction `utils.compute_auc` et l'utiliser dans la fonction `compute_auc_both_models` pour calculer l'ASC des courbes ROC curves obtenues en 2. et 3. .

In [ ]:
solution.compute_auc_both_models()

5. Compléter la fonction `compute_auc_untrained_model` qui calculera la ASC de votre modèle non entraîné.

    Remarque: Etant donnée qu'on réalise une prédiction multi-tâche, les sorties de votre modèle et les cibles seront de tailles (taille du batch, nombre de cibles). Aplatir (*flatten*) ces matrices de tel sorte à avoir deux vecteurs, chaun de taille "(taille du batch) x (nombre de cibles)". Ensuite, procéder au calcul de l'ASC comme si vous auviez une seule tâche de prédiction.

In [ ]:
solution.compute_auc_untrained_model(model, basset_dataloader_test, device)

# **Question 4** (Entrainer le réseau)
(20 points) Écrivons à présent la boucle d'entraînement pour Basset.

1. On rappelle que pour chaque entrée nous avons une cible par expériences, et chaque cible est binaire. Écrire la fonction `get_critereon` qui prend la fonction `loss` appropriée:
Indice: vous pouvez consulter la [documentation PyTorch](https://pytorch.org/docs/stable/nn.html#loss-functions)

In [ ]:
critereon = solution.get_critereon()

2. Compléter la boucle d'entrainement en remplissant les parties manquantes du code dans les fonctions`train_loop` et `valid_loop` dans `solution.py`.

Les deux boucles doivent retourner le coût et l'ASC (calculer de la même façon qu'à la question précédente). Ces deux valeurs doivent être retournées par chaque fonction (référez-vous au docstring pour plus de détails).

In [ ]:
optimizer = optim.Adam(list(model.parameters()), lr=learning_rate, betas=(0.9, 0.999))

valid_score_best = 0
patience = 2
num_epochs = 5  # you don't need to train this for that long!

for e in range(num_epochs):
    train_score, train_loss = solution.train_loop(model, basset_dataloader_train, device, optimizer, criterion)
    valid_score, valid_loss = solution.valid_loop(model, basset_dataloader_valid, device, optimizer, criterion)

    print('epoch {}: loss={:.3f} score={:.3f}'.format(e,
                                                      valid_loss,
                                                      valid_score))

    if valid_score > valid_score_best:
        print('Best score: {}. Saving model...'.format(valid_score))
        torch.save(model, 'model_params.pt')
        valid_score_best = valid_score
    else:
        patience -= 1
        print('Score did not improve! {} <= {}. Patience left: {}'.format(valid_score,
                                                                          valid_score_best,
                                                                          patience))
    if patience == 0:
        print('patience reduced to 0. Training Finished.')
        break

# **Question 5 (Interprétation modèle)**

(30 points) Pour le déployer de modèles d'apprentissage profond dans le monde réel, il est *crucial* de vérifier que nos modèles ont effectivement appris ce qu'ils étaient supposés apprendre.
Dans cette exercice, nous répliquerons une partie de la Figure 3b de [Basset](https://pubmed.ncbi.nlm.nih.gov/27197224/).

Dans la génétique, il existe des *motifs* ADN de référence: il s'agit de courtes séquences qui apparaissent tout au long de notre ADN, et dont la fonction est bien documentée. On s'attend à ce que les noyaux (*kernel*) de la première couche convolutionelle apprennent à identifier certains de ces motifs afin de résoudre la tâche de prédiction.

**Veuillez soumettre vos réponses à cette question en une seule page PDF !**

1. D'abord, vérifions que notre modèle a effectivement bien appris. Générer la courbe ROC et calculer l'ASC de votre modèle après entrainement. Comparer les courbes ROC et les ASC avant et après entrainement avec les modèles simulés précédemment. Que remarquez-vous ?

2. Nous représentons des motifs par des matrics poids position (MPPs). Il s'agit d'une matrice de taille $4 \times$ la longueur de la séquence, où la $(i,j)$ème entrée correspond à la fréquence à laquelle la $i$ème paire de base apparait à la $j$ème position. Ouvrir la MPP du motif CTCF, que vous pouvez trouver dans `MA0139.1.jaspar`. Normaliser cette matrice afin que chaque colonne somme à $1$. 

3. Dans la section "Methods" de l'[article](https://pubmed.ncbi.nlm.nih.gov/27197224/)(page 998), les auteurs décrivent comment ils convertissent chacun des $300$ noyaux (*kernel/filter*) en PMMs normalisées.
D'abord, pour chaque noyau, ils déterminent la valeur maximumale activée à travers le *dataset* (vous pouvez utiliser un sous-ensemble de l'ensemble test ici). Calculer ces valeurs.

4. Après, ils comptent les occurences des paires de bases dans l'ensemble des séquences qui active le noyau à au moins la moitié de sa valeur maximale (calculée dans la question précédente).

Note: Vous devez utiliser `torch.functional.unfold`.

5. Vérifier si certaines des 300 PPMs obtenues de vos noyaux convolutionels sont similai à la PPM du motif CTCF. Vous pouvez mesurer la similarité à l'aide du *Pearson Correlation Coefficient*. Générer une visualisation de la PPM du motif accompagnée de la PPM la plus similaire que le réseau a apprise.

**Quelques points qui peuvent vous aider avec cette question**
* J'ai remarqué que la vitesse des instances de Google Colab varie considérablement (probablement le GPU qui vous est attribué). Si votre code s'exécute très lentement, vous pouvez le redémarrer jusqu'à ce que vous obteniez une instance plus rapide.
* Vous êtes libre de changer la fonction de notation lors de l'entraînement (j'utilise la même AUC mais j'ai changé les incréments de seuils de 0,05 à 0,01 et j'ai constaté que le score augmentait plus visiblement).